# **Localización y clasificación de objetos en video con YOLO**

### **Instalar onnx.**
Herramienta para modelos de Machine Learning

In [ ]:
!pip install onnx

### **Librerías.**

In [1]:
import numpy as np
import time
import cv2
import matplotlib.pyplot as plt

En particular usaremos extensivamente el paquete y funciones ```cv2.dnn``` que permiten implementar yolo de una manera muy fácil y sencilla en cualquier aplicación escrita en Python donde se trabaje con imágenes. 

Primero necesitamos cargar los nombres de las clases en las que se ha entrenado la red. Ten en cuenta que la red que utilicemos solo podrá reconocer las clases en las que se ha formado.

In [2]:
def readLabels(path):
    try:
        labels = open(path).read().strip().split("\n")
        return labels
    except:
        print("Error: Could not read file")

In [3]:
labelsPath = "./yolo-coco/coco.names"
LABELS = readLabels(labelsPath)

In [4]:
LABELS[0:5]

['person', 'bicycle', 'car', 'motorbike', 'aeroplane']

Ahora generaremos un color para cada clase

In [5]:
# Cambiar semilla de números pseudo-aleatorios
np.random.seed(42)
# Color para cada clase
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

Observemos el color de cada clase

In [6]:
COLORS[0:5]

array([[102, 220, 225],
       [ 95, 179,  61],
       [234, 203,  92],
       [  3,  98, 243],
       [ 14, 149, 245]], dtype=uint8)

In [18]:
# Función para hacer una imagen cuadrada
def format_yolov5(frame):
    row, col, _ = frame.shape
    _max = max(col, row)
    result = np.zeros((_max, _max, 3), np.uint8)
    result[0:row, 0:col] = frame
    return result

# Cambiamos la imagen a un cuadrado de 640x640 píxeles y hacemos un forward
def forward(net, input_image):
    blob = cv2.dnn.blobFromImage(input_image, 1 / 255.0, (640, 640), swapRB=True, crop=False)
    net.setInput(blob)
    start = time.time()
    layerOutputs = net.forward()
    end = time.time()
    return layerOutputs, start, end

# Seleccionador de cuadros delimitadores
def bounding_boxes(layerOutputs, input_image):
    class_ids, confidences, boxes = [], [], []
    output_data = layerOutputs[0]
    
    image_width, image_height, _ = input_image.shape
    x_factor = image_width / 640
    y_factor =  image_height / 640
    
    # Hacemos un bucle para cada una de las detecciones
    for r in range(len(output_data)):
        row = output_data[r]
        confidence = row[4]

        if confidence >= .4:

            classes_scores = row[5:] 
            _, _, _, max_indx = cv2.minMaxLoc(classes_scores)
            class_id = max_indx[1]
            
            if (classes_scores[class_id] > .25):
                """Escalar las coordenadas del cuadro delimitador hacia atrás en relación con el
                tamaño de la imagen, teniendo en cuenta que YOLO en realidad
                devuelve las coordenadas del centro (x, y) del límite
                cuadro seguido del ancho y alto de los cuadros."""
                
                confidences.append(confidence)
                class_ids.append(class_id)

                x, y, w, h = row[0].item(), row[1].item(), row[2].item(), row[3].item() 
                
                left = int((x - 0.5 * w) * x_factor)
                top = int((y - 0.5 * h) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                
                box = np.array([left, top, width, height])
                boxes.append(box)
                
    return class_ids, confidences, boxes

# Dibujar cuadros
def draw_boxes(boxes, confidences, class_ids, image):
    # aplicar supresión no máxima para suprimir cuadros delimitadores débiles y superpuestos
    idxs = cv2.dnn.NMSBoxes(boxes, confidences, 0.6, 0.2)
    # asegúrese de que exista al menos una detección
    if len(idxs) > 0:
        # bucle sobre los índices que estamos manteniendo
        for i in idxs.flatten():
            # extraer las coordenadas del cuadro delimitador
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            # dibuje un rectángulo de cuadro delimitador y una etiqueta en la imagen
            color = [int(c) for c in COLORS[class_ids[i]]]
            cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[class_ids[i]], confidences[i])
            cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

Cargamos nuestro modelo de Yolo

In [12]:
modelPath = "./yolo-coco/yolov5s.onnx"

print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNet(modelPath)

[INFO] loading YOLO from disk...


Ahora cargamos el video que queremos analizar. 

In [13]:
video = cv2.VideoCapture('./videos/test/soccer.mp4')
writer  = None

Determinamos el número total de frames en el video.

In [14]:
import imutils
frames = cv2.cv.CV_CAP_PROP_FRAME_COUNT if imutils.is_cv2() \
		else cv2.CAP_PROP_FRAME_COUNT
total = int(video.get(frames))

Recorremos los frames del video y analizamos frame por frame, guardamos las etiquetas y marcos de los objetos detectados en el mismo video, generando otro.

In [19]:
while True:
	# Leer el siguiente frame
	(r, frame) = video.read()
	# Si el frame no fue leido, salimos del loop
	if not r:
		break

	# Construimos un blob desde el marco de entrada y luego realizamos un envío de este
	# hacía el detector de objetos (YOLO). Para construir los marcos de salida, y las probabilidades
	# Necesitamos cambiar el tamaño de la imagen a un cuadrado de 416x416 píxeles y hacer un pase hacia adelante a través de la red.

	input_frame = format_yolov5(frame)
	
	layerOutputs, start, end = forward(net, input_frame)

	# Inicializamos la lista de marcos de salida detectados, las clases y las probabilidades respectivamente
	class_ids, confidences, boxes = bounding_boxes(layerOutputs, input_frame)

	draw_boxes(boxes, confidences, class_ids, frame)

	# Si el video writer no está inicializado, inicializamos uno
	if writer is None:
		# Inicializamos nuestro video writer
		format = cv2.VideoWriter_fourcc(*"mp4v")
		writer = cv2.VideoWriter("./videos/detection/video_soccer_detection.mp4", format, 30, (frame.shape[1], frame.shape[0]), True)

	# Guardamos el video con las etiquetas y los marcos de salida
	writer.write(frame)
writer.release()
video.release()